# MUST RUN IN JUPYTER NOTEBOOK - RUN COMMAND "jupyter notebook quality_control.ipynb"

# BEFORE RUNNING:
## Create a onetime copy of the original JSON you will be using and use that as path. Otherwise changes will be irreversible

## If running for the FIRST TIME: uncomment #create_csv() in line 35 in the 3rd code cell. this will create a checkpoint csv to save your spot. BE SURE TO RECOMMENT OUT.

In [1]:
from IPython.display import HTML, clear_output
import json
import time
import pandas as pd

In [2]:
JSON_PATH = "../../data/survai_data/var_annot_jsons/13-07-2022-14_59_00_master.json"
with open(JSON_PATH) as json_file:
    data = json.load(json_file)

df = pd.read_csv('qc_checkpoints_donotdelete.csv', index_col='label')


#YO JACKSON REMOVE THE CODE BELOW THIS WHEN DONE DAWG 
for key, value in data.items():
    try:
        data[key]['tags']
    except:
        data[key]['tags'] = {"clarity_level": "", "bad_egg": False}
    
with open(f"ADJUSTED.json", "w") as outfile:
    json.dump(data, outfile)

In [3]:
def play_vid(html):
    display(HTML(html))
    
def save_changes(json_object, vid_id, new_start, new_end, tags):
    
    # save vid segment changes
    json_object[vid_id]['annotations']['segment'][0] = new_start
    json_object[vid_id]['annotations']['segment'][1] = new_end
    json_object[vid_id]['duration'] = new_end - new_start
    json_object[vid_id]['tags'] = tags
    
    #overwrite json object with changes
    with open(JSON_PATH, 'w') as outfile:
        json.dump(json_object, outfile)
    
    #checkpoint save spot into csv
    json_label = json_object[vid_id]['annotations']['label']
    df.at[json_label, 'last_id'] = vid_id
    df.to_csv('qc_checkpoints_donotdelete.csv')
        
        
def create_csv():
    "ONLY MEANT TO BE RAN ONCE, OR WHEN NEW LABEL IS CREATED"
    
    df = pd.DataFrame(columns=['label', 'last_id'])
    
    for key in data.keys():
        if data[key]['annotations']['label'] not in df['label'].unique():
            row = {'label': data[key]['annotations']['label'], 'last_id': None}
            df = df.append(row, ignore_index=True)
    df = df.set_index('label')
    df.to_csv("qc_checkpoints_donotdelete.csv")
    print(df)

    """IF RUNNING FOR THE FIRST TIME, UNCOMMENT THE FOLLOWING CODE. 
    
    !!!IMMEDIATELY RECOMMENT OUT AFTER USE!!!
    """
#create_csv()

In [5]:
initial_input = input("type label (case sensitive) you want to go through.")
selected_label = initial_input.split(" ")[0]
breakout = False

all_label_ids = []

# create list of all keys with matching labels if not bad egg
for key in data.keys():
    if data[key]['annotations']['label'] == selected_label:
        if data[key]['tags']['bad_egg'] == False:
            all_label_ids.append(key)
        else:
            continue
            
# find spot left off at
for i, vid_id in enumerate(all_label_ids):
    
    # if vid id is same as checkpoint
    if vid_id == df.at[selected_label, 'last_id']:
        starting_index = i
        break
    else: 
        starting_index = 0


        
        
# iterate all videos
while True:
    
    key = all_label_ids[starting_index]
    
 
    key_str = key[:11]
    start = int(data[key]['annotations']['segment'][0])
    end = int(data[key]['annotations']['segment'][1])
    duration = int(data[key]['duration'])
    label = data[key]['annotations']['label']
    tags= data[key]['tags']
    cl_tag = data[key]['tags']['clarity_level']
    be_tag = data[key]['tags']['bad_egg']
    adjusting_annot = True
    is_changing = False
    
    
    if breakout == True:
        text_file = open("log_records.txt", "r").read()
        print(text_file)
        time.sleep(3)
        clear_output() 
        break
        
    if cl_tag != '':
        untagged = False
    else:
        untagged = True 
        
        
    #iterate same video until ready
    while adjusting_annot == True:
            
        clear_output(wait=True) 
        html = f'<iframe width="560" height="315" \
            src="https://www.youtube.com/embed/{key_str}?autoplay=1&start={start}&end={end}&loop=1" \
            title="YouTube video player" \
            frameborder="0" \
            allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" \
            allowfullscreen></iframe>'
        play_vid(html)
        if be_tag == True:
            print("BADEGG BADEGG BADEGG BADEGG BADEGG BADEGG BADEGG BADEGG")
        if is_changing == True:
            print(f"index:{starting_index}/{len(all_label_ids)}","label:", label, "start:", start, "end:", end, f"original:{int(data[key]['annotations']['segment'][0])} - {int(data[key]['annotations']['segment'][1])}", "tag:", cl_tag)
        else:
            print(f"index:{starting_index}/{len(all_label_ids)}", "label:", label, "start:", start, "end:", end, "tag:", cl_tag)
        time.sleep(1)
        

        
        input_step_2 = input("r=restartvid, (s or e)(num)=change start or end time,xxx=badegg(delete), xxxr=reversedelete, cl (e,m,h)=claritylevel, exit=exit")
        
        # restart video
        if input_step_2 == 'r':
            continue
            
        # if changing start value    
        if 's' in input_step_2 and len(input_step_2) > 1 and len(input_step_2) < 4:
            is_changing = True
            change_int = int(input_step_2[1:])
            
            if start + change_int >= 0:
                start += change_int
                continue
            else:
                print("ayo?")
                time.sleep(5)
                continue

        # if changing end value
        print(len(input_step_2))
        if 'e' in input_step_2 and len(input_step_2) > 1 and len(input_step_2) < 4:
            is_changing = True
            change_int = int(input_step_2[1:])
            end += change_int
            continue
                
        # delete annotation
        if input_step_2 == "xxx":
            data[key]['tags']['bad_egg'] = True
            be_tag = data[key]['tags']['bad_egg']
            untagged = False
            
        if input_step_2 == "xxxr":
            data[key]['tags']['bad_egg'] = False
            be_tag = data[key]['tags']['bad_egg']
              
        # tag video
        if "cl" in input_step_2:
            command = input_step_2.split(" ")[1]
            if command == 'e':
                data[key]['tags']['clarity_level'] = 'easy'
                cl_tag = data[key]['tags']['clarity_level']
                untagged = False
            elif command == 'm':
                data[key]['tags']['clarity_level'] = 'medium'
                cl_tag = data[key]['tags']['clarity_level']
                untagged = False
            elif command == 'h':
                data[key]['tags']['clarity_level'] = 'hard'
                cl_tag = data[key]['tags']['clarity_level']
                untagged = False
            else:
                html = f'<iframe width="560" height="315" \
                src="https://www.youtube.com/embed/D2_r4q2imnQ?autoplay=1&loop=1" \
                title="YouTube video player" \
                frameborder="0" \
                allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" \
                allowfullscreen></iframe>'
                play_vid(html)
                time.sleep(5)
                continue
                
                
        # if going to next video  
        if input_step_2 == 'n':
            save_changes(data, key, start, end, tags)
            if untagged == True:
                print('no tag on video')
                time.sleep(3)
                continue
            else:
                starting_index += 1
                adjusting_annot = False
            
        # if going to previous video  
        if input_step_2 == 'p':
            save_changes(data, key, start, end, tags)
            starting_index -= 1
            adjusting_annot = False
                
        #exit app
        if input_step_2 == 'exit':
            save_changes(data, key, start, end, tags)
            breakout = True
            break
            
            


type label (case sensitive) you want to go through.nothing


KeyError: 'nothing'